In [ ]:
import pandas as pd
import pyslim
import tskit
import numpy as np

# Parallelizing SLiM simulations in a phylogenetic tree

In [ ]:
path_to_tsv = "./phylo.tsv"
path_to_make = "./parallel_sims.make"
path_to_slimscript = "./simulate_branch.slim"
path_to_unioned = "./unioned.trees"

## Simulating the branches

### Creating the Makefile to run simulations

In [ ]:
# Reading the phylogeny data frame
df = pd.read_csv(path_to_tsv, sep="\t")
df = df.fillna('')

## Creating intermediate tree sequences filenames
df["infile"] = df.parent + ".trees"
df["outfile"] = df.child + ".trees"
df.loc[df["infile"]==".trees", "infile"] = ""
df["is_leaf"] = ~df.child.isin(df.parent) # setting nodes that are never parents as leaves

In [ ]:
# Writing a Makefile 
f = open(path_to_make, "w")
print(f"all: {' '.join(df.outfile.to_list())}\n", file=f)
for i, row in df.iterrows():
    print(f"{row.outfile}: {row.infile} {path_to_slimscript}", file=f)
    print(f"\tslim -s 123 -d \"infile='{row.infile}'\" -d popsize={row.popsize} "
          f"-d \"popname=\'{row.child}\'\" "
          f"-d num_gens={row.edgelen} " f"-d \"outfile='{row.child}.trees'\" "
          f"{path_to_slimscript}\n",
          file=f)
f.close()

### Running the simulations in parallel

In [ ]:
%%bash
make -f parallel_sims.make -j 3

## Putting it all together: unioning the tree sequences

In [ ]:
def match_nodes(other, ts, split_time):
    node_mapping = np.full(other.num_nodes, -1, dtype='int')
    ts_ids = np.array([n.metadata["slim_id"] for n in ts.nodes()])
    other_ids = np.array([n.metadata["slim_id"] for n in other.nodes()])
    alive_before_split1 = (other.tables.nodes.time >= split_time)
    shared_id = np.isin(other_ids, ts_ids)
    both = np.logical_and(alive_before_split1, shared_id)
    sorted_ids = np.argsort(ts_ids)
    matches = np.searchsorted(
        ts_ids,
        other_ids[both],
        side='left',
        sorter=sorted_ids
    )
    node_mapping[both] = sorted_ids[matches]
    return node_mapping

In [ ]:
def union_children(parent, df, merged):
    print(f"Going in: {parent}")
    child_rows = df[df.parent == parent]
    assert (len(child_rows) == 2) or (len(childs) == 0)
    if len(child_rows) == 2:
        children = [row.child for _, row in child_rows.iterrows()]
        for child in children:
            if child not in merged:
                union_children(child, df, merged)
        split_time = merged[children[0]]["depth"]
        assert split_time == merged[children[1]]["depth"] # ultrametric
        print(f'Unioning: {children}, Split time: {split_time}')
        ts0 = merged[children[0]]["ts"]
        ts1 = merged[children[1]]["ts"]
        node_map = match_nodes(ts1, ts0, split_time)
        tsu = ts0.union(ts1, node_map, check_shared_equality=True)
        # the time from tip to start of simulation is split_time plus the
        # length of the edge
        parent_edgelength = df[df.child==parent].edgelen.item()
        merged[parent] = {
            "ts": tsu,
            "depth": split_time + parent_edgelength,
            "children": merged[children[0]]["children"] + merged[children[1]]["children"]
        }

In [ ]:
pops = ['A', 'B', 'C']
A, B, C = (tskit.load(f"{x}.trees") for x in pops)
nm = match_nodes(B, A, split_time=50)
AB = A.union(B, nm)
nm = match_nodes(C, AB, split_time=200)
ABC = AB.union(C, nm)

In [ ]:
alive = np.where(np.isclose(ABC.tables.nodes.time, 0))[0]
pop_ids = {}
for pop in ABC.populations():
    if pop.metadata is not None:
        pop_ids[pop.metadata['name']] = pop.id

for name in pops:
    pop_samples = ABC.samples(pop_ids[name])
    n_samples = sum(np.isin(pop_samples, alive)) // 2
    print(f"Union-ed tree sequence has {n_samples} samples in population {name},\n"
          f"\tand we specified {df[df.child==name].popsize.item()} individuals in our simulations.")
    assert n_samples == df[df.child==name].popsize.item()